In [1]:
import torch
import random
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import argparse,os,time
import os
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
num_gpus=4

In [2]:
data = pd.read_csv('train_augmented.csv', index_col=0)
val_data=pd.read_csv('val_augmented.csv', index_col=0)

In [3]:
y_data=data["digit"].values
y_data_val=val_data["digit"].values

In [4]:
y_data.shape

(81900,)

In [5]:
x_data=data.loc[:,"0":"783"].values
x_data_val=val_data.loc[:,"0":"783"].values

In [6]:
x_data_train=x_data
x_data_train.shape

(81900, 784)

In [7]:
x_data_test=x_data_val
x_data_test.shape

(20500, 784)

In [8]:
x_data_train=x_data_train/x_data_train.max()
x_data_test=x_data_test/x_data_test.max()
x_data_train.shape

(81900, 784)

In [9]:
class CustomDataset(Dataset):
    def __init__(self,x_dat,y_dat):
        x = x_dat
        y = y_dat
        self.len = x.shape[0]
        y=y.astype('int')
        x=x.astype('float32')
        self.x_data = torch.tensor(x)
        self.y_data = torch.tensor(y)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [10]:
batch_size=256
train_dataset = CustomDataset(x_data_train,y_data)
train_loader = DataLoader(dataset=train_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)
test_dataset = CustomDataset(x_data_test,y_data_val)
test_loader = DataLoader(dataset=test_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)

In [11]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer_1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_1 = nn.ReLU()
        self.conv2_bn1 = nn.BatchNorm2d(64)
        
        self.layer_2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_2 = nn.ReLU()
        self.conv2_bn2 = nn.BatchNorm2d(64)
        
        self.layer_3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_3 = nn.ReLU()
        self.conv2_bn3 = nn.BatchNorm2d(64)
        
        self.max_1=nn.MaxPool2d(2,2)
        
        self.layer_4 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_4 = nn.ReLU()
        self.conv2_bn4 = nn.BatchNorm2d(128)

        self.layer_5 = nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_5 = nn.ReLU()
        self.conv2_bn5 = nn.BatchNorm2d(128)
        
        self.layer_6 = nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_6 = nn.ReLU()
        self.conv2_bn6 = nn.BatchNorm2d(128)
        self.max_2=nn.MaxPool2d(2,2)
        
        self.layer_7 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_7 = nn.ReLU()
        self.conv2_bn7 = nn.BatchNorm2d(256)
        
        self.layer_8 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_8 = nn.ReLU()
        self.conv2_bn8 = nn.BatchNorm2d(256)
        
        self.layer_9 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_9 = nn.ReLU()
        self.conv2_bn9 = nn.BatchNorm2d(256)
        
        self.max_3=nn.MaxPool2d(2,2)
        
        self.layer_10 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_10 = nn.ReLU()
        self.conv2_bn10 = nn.BatchNorm2d(512)
        
        self.layer_11 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_11 = nn.ReLU()
        self.conv2_bn11 = nn.BatchNorm2d(512)
        
        self.layer_12 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_12 = nn.ReLU()
        self.conv2_bn12 = nn.BatchNorm2d(512)
        
        self.max_4=nn.MaxPool2d(2,2)
        
        self.layer_13 = nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_13 = nn.ReLU()
        self.conv2_bn13 = nn.BatchNorm2d(1024)
        
        self.layer_14 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_14 = nn.ReLU()
        self.conv2_bn14 = nn.BatchNorm2d(1024)
        
        self.max_5=nn.MaxPool2d(2,2)
        
        self.layer_15 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=1,
                                 stride=1)
        self.act_15 = nn.ReLU()
        self.conv2_bn15 = nn.BatchNorm2d(1024)
        
        self.layer_16 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_16 = nn.ReLU()
        self.conv2_bn16 = nn.BatchNorm2d(1024)
        
        self.layer_17 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=1,
                                 stride=1)
        self.act_17 = nn.ReLU()
        self.conv2_bn17 = nn.BatchNorm2d(1024)
        
        

        
        self.fc_layer_1 = nn.Linear(49*1024,1000)
        self.act_18 = nn.ReLU()
        
        self.bnm1=nn.BatchNorm1d(1000)
        
        self.fc_layer_2 = nn.Linear(1000,1000)
        self.act_19 = nn.ReLU()
        
        self.bnm2=nn.BatchNorm1d(1000)
        
        self.fc_layer_3 = nn.Linear(1000,100)
        self.act_20 = nn.ReLU()
        
        self.bnm3=nn.BatchNorm1d(100)
        
        self.fc_layer_4 = nn.Linear(100,10)
        self.act_21 = nn.ReLU()

        
        
        

    def forward(self, x):
        x = x.view(batch_size//num_gpus,1,28,28)
        out = self.layer_1(x)
        out = self.act_1(out)
        for module in list(self.modules())[2:-11]:
            out = module(out)
        out = out.view(batch_size//num_gpus,-1)
        for module in list(self.modules())[-11:]:
            out = module(out)
        return out

In [12]:
model=nn.DataParallel(Model().cuda())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0.001)

In [13]:
trn_loss_list = []
val_loss_list = []
total_epoch=200
model_char="minloss"
model_name=""
patience=5
start_early_stop_check=0
saving_start_epoch=10

for epoch in range(total_epoch):
    trn_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del output
    with torch.no_grad():
        val_loss = 0.0
        cor_match = 0
        for j, val in enumerate(test_loader):
            val_x, val_label = val
            if torch.cuda.is_available():
                val_x = val_x.cuda()
                val_label =val_label.cuda()
            val_output = model(val_x)
            v_loss = criterion(val_output, val_label)
            val_loss += v_loss
            _, predicted=torch.max(val_output,1)
            cor_match+=np.count_nonzero(predicted.cpu().detach()==val_label.cpu().detach())
    del val_output
    del v_loss
    del predicted
    
    
    
    trn_loss_list.append(trn_loss/len(train_loader))
    val_loss_list.append(val_loss/len(test_loader))
    val_acc=cor_match/(len(test_loader)*batch_size)
    now = time.localtime()
    print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))

    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | val accuracy: {:.4f}% \n".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), val_loss / len(test_loader), val_acc*100
            ))
    
    
    if epoch+1>2:
        if val_loss_list[-1]>val_loss_list[-2]:
            start_early_stop_check=1
    else:
        val_loss_min=val_loss_list[-1]
        
    if start_early_stop_check:
        early_stop_temp=val_loss_list[-patience:]
        if all(early_stop_temp[i]<early_stop_temp[i+1] for i in range (len(early_stop_temp)-1)):
            print("Early stop!")
            break
            
    if epoch+1>saving_start_epoch:
        if val_loss_list[-1]<val_loss_min:
            if os.path.isfile(model_name):
                os.remove(model_name)
            val_loss_min=val_loss_list[-1]
            model_name="Custom_model_"+model_char+"_{:.3f}".format(val_loss_min)
            torch.save(model, model_name)
            print("Model replaced and saved as ",model_name)

2020/09/24 11:19:57
epoch: 1/200 | trn loss: 1.2267 | val loss: 0.6920 | val accuracy: 77.3242% 

2020/09/24 11:21:05
epoch: 2/200 | trn loss: 0.4517 | val loss: 0.4464 | val accuracy: 85.7812% 

2020/09/24 11:22:14
epoch: 3/200 | trn loss: 0.2869 | val loss: 0.3732 | val accuracy: 88.4033% 

2020/09/24 11:23:24
epoch: 4/200 | trn loss: 0.2185 | val loss: 0.3229 | val accuracy: 90.3857% 

2020/09/24 11:24:34
epoch: 5/200 | trn loss: 0.1817 | val loss: 0.3481 | val accuracy: 90.2490% 

2020/09/24 11:25:44
epoch: 6/200 | trn loss: 0.1616 | val loss: 0.3693 | val accuracy: 90.5811% 

2020/09/24 11:26:54
epoch: 7/200 | trn loss: 0.1344 | val loss: 0.3107 | val accuracy: 91.3135% 

2020/09/24 11:28:04
epoch: 8/200 | trn loss: 0.1252 | val loss: 0.2880 | val accuracy: 92.2559% 

2020/09/24 11:29:14
epoch: 9/200 | trn loss: 0.1110 | val loss: 0.3156 | val accuracy: 92.1631% 

2020/09/24 11:30:23
epoch: 10/200 | trn loss: 0.1022 | val loss: 0.3090 | val accuracy: 91.8799% 

2020/09/24 11:31:33

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model replaced and saved as  Custom_model_minloss_0.290
2020/09/24 11:32:43
epoch: 12/200 | trn loss: 0.0878 | val loss: 0.2990 | val accuracy: 92.2510% 

2020/09/24 11:33:53
epoch: 13/200 | trn loss: 0.0866 | val loss: 0.2707 | val accuracy: 92.9492% 

Model replaced and saved as  Custom_model_minloss_0.271
2020/09/24 11:35:04
epoch: 14/200 | trn loss: 0.0844 | val loss: 0.2873 | val accuracy: 92.3828% 

2020/09/24 11:36:14
epoch: 15/200 | trn loss: 0.0804 | val loss: 0.2631 | val accuracy: 92.9053% 

Model replaced and saved as  Custom_model_minloss_0.263
2020/09/24 11:37:24
epoch: 16/200 | trn loss: 0.0832 | val loss: 0.2509 | val accuracy: 93.3252% 

Model replaced and saved as  Custom_model_minloss_0.251
2020/09/24 11:38:34
epoch: 17/200 | trn loss: 0.0728 | val loss: 0.2574 | val accuracy: 93.3008% 

2020/09/24 11:39:44
epoch: 18/200 | trn loss: 0.0755 | val loss: 0.2825 | val accuracy: 92.6855% 

2020/09/24 11:40:54
epoch: 19/200 | trn loss: 0.0768 | val loss: 0.2658 | val accur

2020/09/24 13:05:05
epoch: 91/200 | trn loss: 0.0407 | val loss: 0.2918 | val accuracy: 93.7451% 

2020/09/24 13:06:15
epoch: 92/200 | trn loss: 0.0441 | val loss: 0.3133 | val accuracy: 93.0273% 

2020/09/24 13:07:26
epoch: 93/200 | trn loss: 0.0425 | val loss: 0.2270 | val accuracy: 94.4580% 

2020/09/24 13:08:36
epoch: 94/200 | trn loss: 0.0368 | val loss: 0.2593 | val accuracy: 94.3115% 

2020/09/24 13:09:46
epoch: 95/200 | trn loss: 0.0423 | val loss: 0.2661 | val accuracy: 93.7695% 

2020/09/24 13:10:56
epoch: 96/200 | trn loss: 0.0414 | val loss: 0.2369 | val accuracy: 94.0820% 

2020/09/24 13:12:07
epoch: 97/200 | trn loss: 0.0402 | val loss: 0.2475 | val accuracy: 94.3408% 

2020/09/24 13:13:17
epoch: 98/200 | trn loss: 0.0427 | val loss: 0.2524 | val accuracy: 94.0576% 

2020/09/24 13:14:28
epoch: 99/200 | trn loss: 0.0409 | val loss: 0.2494 | val accuracy: 94.0771% 

2020/09/24 13:15:38
epoch: 100/200 | trn loss: 0.0432 | val loss: 0.2378 | val accuracy: 94.2480% 

2020/09/2

2020/09/24 14:42:29
epoch: 174/200 | trn loss: 0.0347 | val loss: 0.2518 | val accuracy: 94.3262% 

2020/09/24 14:43:40
epoch: 175/200 | trn loss: 0.0411 | val loss: 0.2714 | val accuracy: 93.8379% 

2020/09/24 14:44:50
epoch: 176/200 | trn loss: 0.0445 | val loss: 0.2947 | val accuracy: 93.7012% 

2020/09/24 14:46:00
epoch: 177/200 | trn loss: 0.0360 | val loss: 0.2685 | val accuracy: 93.8037% 

2020/09/24 14:47:10
epoch: 178/200 | trn loss: 0.0339 | val loss: 0.2385 | val accuracy: 94.3457% 

2020/09/24 14:48:21
epoch: 179/200 | trn loss: 0.0396 | val loss: 0.2670 | val accuracy: 93.9355% 

2020/09/24 14:49:31
epoch: 180/200 | trn loss: 0.0393 | val loss: 0.2668 | val accuracy: 94.2676% 

2020/09/24 14:50:42
epoch: 181/200 | trn loss: 0.0323 | val loss: 0.2470 | val accuracy: 94.3848% 

2020/09/24 14:51:52
epoch: 182/200 | trn loss: 0.0390 | val loss: 0.3159 | val accuracy: 93.2666% 

2020/09/24 14:53:03
epoch: 183/200 | trn loss: 0.0394 | val loss: 0.2930 | val accuracy: 93.3643% 



In [14]:
torch.save(model, "Custom_model_fin")
print("model saved complete")

model saved complete
